In [ ]:
import src
import keras.backend as K
import os
import numpy as np
import sys
import re
import math
import io
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from  matplotlib.animation import FuncAnimation
from matplotlib import colors
from netCDF4 import Dataset
from IPython.display import clear_output
#data folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/data')
#forces CPU usage
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #"" or "-1" for CPU, "0" for GPU
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
def sample_images(epoch, gan_test, gan_test_truth, past_input):
    n = 5
    test_batch = gan_test[:n]
    test_truth = gan_test_truth[:n]
    gen_imgs = generator.predict(test_batch)
    plot_range = past_input 
    fig, axs = plt.subplots(n, plot_range+2, figsize=(16, 16))
    for i in range(n):
        vmax = np.max([np.max(test_batch[i]), np.max(test_truth[i])])
        vmin = 0
        for j in range(plot_range):
            im = axs[i,j].imshow(test_batch[i, :,:,j], vmax=vmax,vmin=vmin)
            axs[i,j].axis('off')
            src.colorbar(im)
            axs[i,j].set_title("Frame t"+str([-past_input+1+j if j < past_input-1 else ""][0]))
        im2 = axs[i,-2].imshow(test_truth[i, :,:,0], vmax=vmax, vmin=vmin)
        axs[i,-2].axis('off')
        src.colorbar(im2)                
        axs[i,-2].set_title("Frame t+1")
        im3 = axs[i,-1].imshow(gen_imgs[i, :,:,0], vmax=vmax, vmin=vmin)
        axs[i,-1].axis('off')
        src.colorbar(im3)
        axs[i,-1].set_title("Prediction t+1")
    fig.savefig("Plots/epoch %d.png" % epoch)
    plt.close()

Load dataset.

In [ ]:
past = 4

In [ ]:
train, xval, test = src.load_datasets(past_frames=past)

Split data to inputs and ground truth images.

In [ ]:
gan_train, gan_truth, gan_val, gan_val_truth, gan_test, gan_test_truth = src.split_datasets(
            train[:2000], xval, test, past_frames=past, augment=True)

Make discriminator labels.

In [ ]:
batch_size=64

In [ ]:
# Adversarial ground truths
real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))
#Generator ground truths
g_real = np.ones((batch_size, 1))

Make generator but don't compile.

In [ ]:
generator = src.unet(gan_train.shape[1:], dropout=0, batchnorm=False)

In [ ]:
generator.summary()

Make discriminator and compile.

In [ ]:
discriminator = src.spatial_discriminator(condition_shape=gan_train.shape[1:], dropout = 0.25, batchnorm=True)
discriminator.compile(loss=keras.losses.binary_crossentropy,optimizer=keras.optimizers.SGD(),
                      metrics=[keras.metrics.binary_accuracy])

Inputs and outputs of the GAN.

In [ ]:
frame_t = keras.layers.Input(shape=gan_train.shape[1:])

In [ ]:
frame_t.shape

In [ ]:
generated = generator(frame_t)

In [ ]:
score = discriminator([frame_t, generated])

Freeze discriminator weights.

In [ ]:
discriminator.trainable = False

Compile combined model.

In [ ]:
loss_weights=[0.1,1]

In [ ]:
combined = keras.models.Model(inputs=[frame_t], outputs=[generated, score])

In [ ]:
combined.compile(loss=[src.custom_loss(loss="l1"), keras.losses.binary_crossentropy], optimizer=keras.optimizers.Adam(0.0002, 0.5),
                 loss_weights=loss_weights)

In [ ]:

log = {"g_loss":[],
               "d_loss":[],
               "g_metric":[],
               "d_metric":[],
      "d_loss_real":[],
      "d_loss_fake":[],
      "d_test_real":[],
      "d_test_fake":[]}

Train x epochs.

In [ ]:
epochs = 300

In [ ]:
for epoch in range(epochs):
    discriminator.trainable = True
    idx = np.random.randint(0, gan_truth.shape[0], batch_size)
    real_imgs = gan_truth[idx]
    training_batch = gan_train[idx]
        
    generated_imgs = generator.predict(training_batch) 
    d_loss_real = discriminator.train_on_batch([training_batch, real_imgs], real)
    d_loss_fake = discriminator.train_on_batch([training_batch, generated_imgs], fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
     #for l in discriminator.layers:
     #   weights = l.get_weights()
     #   weights = [np.clip(w, -clip_value, clip_value) for w in weights] 
     #   l.set_weights(weights)
    
    #d_test_real = discriminator.predict([training_batch, real_imgs])
    #d_test_fake = discriminator.predict([training_batch, generated_imgs])
    discriminator.trainable = False    
    idx = np.random.randint(0, gan_train.shape[0], batch_size)
    training_batch = gan_train[idx]
    training_truth = gan_truth[idx]
    
    g_loss = combined.train_on_batch(training_batch, [training_truth, g_real])
    
    if g_loss[1] < 0.12 and loss_weights[0] > 2**(-1):
        loss_weights[0] /= 2
        combined.compile(loss=[src.custom_loss(loss="l1"), keras.losses.binary_crossentropy], optimizer=keras.optimizers.Adam(0.0002, 0.5),
                 loss_weights=loss_weights)
    
    log["g_loss"].append(g_loss)
    log["d_loss_real"].append(d_loss_real[0])
    log["d_loss_fake"].append(d_loss_fake[0])
    #log["d_test_real"].append(d_test_real)
    #log["d_test_fake"].append(d_test_fake)
    log["d_loss"].append(d_loss[0])
    log["d_metric"].append(d_loss[1])
    print(f"\033[1m {epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}]\033[0m"+
                  f"\033[1m[G loss: {g_loss[0]}, G obj.: {g_loss[1]}, G bce.: {g_loss[2]}]\033[0m")
    print(f"real loss: {d_loss_real}, fake loss: {d_loss_fake}")
    #print(f"test real acc.: {np.mean(d_test_real)}, test fake acc.: {np.mean(d_test_fake)}")
    if epoch%10 == 0:
        sample_images(epoch, gan_test, gan_test_truth, past)

In [ ]:
len(log["g_loss"])

In [ ]:
vx, vy = src.optical_flow(gan_train[:,:,:,-2:-1], gan_train[:,:,:,-1:], window_size=4, tau=1e-2)

In [ ]:
#%matplotlib notebook
plt.plot(np.array(log["g_loss"])[:,0], alpha=0.3,c="b")
plt.plot(np.array(log["d_loss"]),alpha=0.3, c="orange")

plt.plot(np.array(log["g_loss"])[:,1], alpha=0.9,c="green", label="L1 objective")

plt.plot(src.smooth(np.array(log["g_loss"])[:,0]),c="b", label="generator")
plt.plot(src.smooth(np.array(log["d_loss"])),c="orange", label="discriminator")
plt.grid()
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend()
#plt.savefig("sGAN_training_curves_r")


## Save network weights and log

In [ ]:
def save_model_params(name, model, log, scores):
    for key in list(scores.keys()):
        log[key] = scores[key]
    np.save("C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/ADL4CV_project/trainings/"+name+"/"+name, log) 
    model.save_weights("C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/ADL4CV_project/trainings/"+name+"/"+name)

In [ ]:
name = "sgan_4-1"

In [ ]:
predictions = generator.predict(gan_test)

In [ ]:
scores = src.calculate_skill_scores(predictions, gan_test_truth, gan_test, 0.3)

In [ ]:
save_model_params(name, combined, log, scores)

Sequence prediction

In [ ]:
new_input = np.concatenate((gan_test[:,:,:,1:],predictions),axis=-1)

In [ ]:
new_predictions = generator.predict(new_input)

In [ ]:
plt.imshow(new_predictions[0,:,:,0])
plt.colorbar()

In [ ]:
np.mean(scores["corr_to_truth"])

In [ ]:
np.mean(scores["corr_to_input"])

In [ ]:
#%matplotlib notebook
plt.imshow(predictions[0,:,:,0], vmin=0)

In [ ]:
#%matplotlib notebook
plt.imshow(gan_test_truth[0,:,:,0], vmin=0)

In [ ]:
scores = src.calculate_skill_scores(predictions, gan_test_truth, gan_test, 0.3)

In [ ]:
np.mean(scores["corr_to_input"])

In [ ]:
np.mean(scores["corr_to_truth"])

In [ ]:
scores.keys()

In [ ]:
np.mean(scores["csi"])

In [ ]:
np.mean(scores["far"])

In [ ]:
np.mean(scores["pod"])

In [ ]:
def wasserstein_loss(y_true, y_pred):
    """Calculates the Wasserstein loss for a sample batch.
    The Wasserstein loss function is very simple to calculate. In a standard GAN, the discriminator
    has a sigmoid output, representing the probability that samples are real or generated. In Wasserstein
    GANs, however, the output is linear with no activation function! Instead of being constrained to [0, 1],
    the discriminator wants to make the distance between its output for real and generated samples as large as possible.
    The most natural way to achieve this is to label generated samples -1 and real samples 1, instead of the
    0 and 1 used in normal GANs, so that multiplying the outputs by the labels will give you the loss immediately.
    Note that the nature of this loss means that it can be (and frequently will be) less than 0."""
    return K.mean(y_true * y_pred)

In [ ]:
def noisy_d_labels(real, fake):
    # idea: https://arxiv.org/pdf/1606.03498.pdf
    batch_size = len(real)
    five_percent = int(0.05*batch_size)
    idx = np.random.randint(0, batch_size, five_percent)
    d_real = np.ones_like(real)
    d_fake = np.zeros_like(fake)
    d_real[idx] = 0
    d_fake[idx] = 1
    return d_real, d_fake